In [1]:
import pandas as pd
import numpy as np
import os
import glob

In [2]:
pd.set_option("display.max_columns", 300)

input = 'input/'
output = 'output/'
mapping = 'mapping/'

# os.makedirs(output, exist_ok=True)

In [3]:
data_input = glob.glob(os.path.join(input+'*.xlsx'))
data_input_con = glob.glob(os.path.join(input+'*.txt'))
data_input

['input\\CAP_data.xlsx']

In [4]:
data_map = glob.glob(os.path.join(mapping+'*csv'))
data_map

['mapping\\RPISIC01_20211031.csv',
 'mapping\\RPISIC01_20211130.csv',
 'mapping\\RPISIC01_20211231.csv',
 'mapping\\RPISIC01_20220131.csv',
 'mapping\\RPISIC01_20220228.csv',
 'mapping\\RPISIC01_20220331.csv',
 'mapping\\RPISIC01_20220430.csv',
 'mapping\\RPISIC01_20220531.csv',
 'mapping\\RPISIC01_20220630.csv',
 'mapping\\RPISIC01_20220731.csv',
 'mapping\\RPISIC01_20220831.csv',
 'mapping\\RPISIC01_20220930.csv',
 'mapping\\RPISIC01_20221031.csv',
 'mapping\\RPISIC01_20221130.csv',
 'mapping\\RPISIC01_20221231.csv']

# loop concat all data -- run 1 time!!!
- if run this, do necessary run again


In [5]:
# appended_data = []

# for file in data_map:
#     data = pd.read_csv(file, encoding="TIS=620", low_memory=False)
#     appended_data.append(data)

# df_con = pd.concat(appended_data)

In [6]:
# df_con[df_con['ACC_ID'] == 9520000566]

In [7]:
# df_con.to_csv(input+"concat_CBS.txt", index=False, encoding="TIS-620", sep="|")

# CAP_Data range between 10/2021 and 12/2022

In [8]:
def capdata_prepare():
    cap_df = pd.read_excel(data_input[0])

    # rename 
    cap_df.rename(columns={'หมายเลขบัตรประจำตัว':'ID_NO'}, inplace=True)
    cap_df.rename(columns={'เลขที่ใบคำขอ':'APPLICATION_IN'}, inplace=True)

    # drop duplicated ID_NO
    cap_df.drop_duplicates(subset=['ID_NO'], inplace=True)

    # string method --> slice text to get month
    cap_df['Month'] = cap_df['APPLICATION_IN'].str.slice(8,12)

    return cap_df

In [9]:
cap_df = capdata_prepare()

In [10]:
cap_df.head()

,As_of,APPLICATION_IN,ID_NO,Month
0,2021-10-31,AP00001-211004-00002,3950500181876,2110
2,2021-10-31,AP00027-211001-00001,3149900277299,2110
3,2021-10-31,AP00027-211004-00002,3140100522762,2110
4,2021-10-31,AP00029-211001-00001,1102700192688,2110
5,2021-10-31,AP00029-211003-00004,1679900168863,2110


In [11]:
cap_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15804 entries, 0 to 24782
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   As_of           15804 non-null  datetime64[ns]
 1   APPLICATION_IN  15804 non-null  object        
 2   ID_NO           15803 non-null  object        
 3   Month           15804 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 617.3+ KB


In [12]:
cap_df['ID_NO'].value_counts()

3950500181876    1
1840800005920    1
3610500158460    1
3101600417527    1
1102200150625    1
                ..
3100500221565    1
2809700020958    1
1240800157601    1
3840600166730    1
1960800066847    1
Name: ID_NO, Length: 15803, dtype: int64

In [13]:
cap_df['As_of'].value_counts().sort_index()

2021-10-31     724
2021-11-30     881
2021-12-31     674
2022-01-30     651
2022-02-28     700
2022-03-31    1042
2022-04-30     681
2022-05-31     858
2022-06-30    1848
2022-07-31    1258
2022-08-31    1569
2022-09-30    1241
2022-10-31    1174
2022-11-30    1388
2022-12-31    1115
Name: As_of, dtype: int64

In [14]:
cap_df['ID_NO'].dtype

dtype('O')

# Filter only "booked ACC_ID" on CBS

In [15]:
def cbsdata_prepare():
    cbs_data_con = pd.read_csv(data_input_con[0], encoding="TIS=620", sep="|", low_memory=False)

    # map ID_NO(CAP) to CBS
    cap_df_ID = cap_df['ID_NO'].unique()
    cbs_data_filt = cbs_data_con[cbs_data_con['ID_NO'].isin(cap_df_ID)]

    # filter ACC_OPEN_DATE >= 10/2021 
    index_accop = cbs_data_filt.columns.get_loc('ACC_OPEN_DATE')
    cbs_data_filt.iloc[:, index_accop] = pd.to_datetime(cbs_data_filt['ACC_OPEN_DATE'], format="%d/%m/%Y")
    cbs_data_filt = cbs_data_filt[cbs_data_filt['ACC_OPEN_DATE'] >= '2021-10-01']

    # sort as_of_date, acc_id
    cbs_data_filt['AS_OF_DATE'] = pd.to_datetime(cbs_data_filt['AS_OF_DATE'], format="%d/%m/%Y")
    cbs_data_filt['ACC_ID'] = cbs_data_filt['ACC_ID'].astype('int64')
    cbs_data_filt.sort_values(by=['ACC_ID', 'AS_OF_DATE'], inplace=True)
    # cbs_data_filt.sort_values(by=['AS_OF_DATE'], inplace=True)

    # change datetime format
    # cbs_data_filt['ACC_OPEN_DATE'] = cbs_data_filt['ACC_OPEN_DATE'].dt.strftime('%d/%m/%Y')
    
    return cbs_data_filt

In [16]:
cbs_data_con = cbsdata_prepare()

C:\Users\650010\AppData\Local\Temp\ipykernel_15936\2253184549.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cbs_data_filt.iloc[:, index_accop] = pd.to_datetime(cbs_data_filt['ACC_OPEN_DATE'], format="%d/%m/%Y")
C:\Users\650010\AppData\Local\Temp\ipykernel_15936\2253184549.py:10: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  cbs_data_filt.iloc[:, index_accop] = pd.to_datetime(cbs_data_filt['ACC_OPEN_DATE'], format="%d/%m/%Y")


In [17]:
print(cbs_data_con.shape)
cbs_data_con.head()

(38182, 63)


,AS_OF_DATE,CUST_ID,CUST_NAME,ACC_ID,BRANCH_CODE,BRANCH_DESC,PROD_TYPE,PROD_TYPE_DESC,CREDIT_LIMIT,LEDGER_BALANCE,CUST_CLASS,DATA_SOURCE,RELIGION,CORE_RELIGION,MUSLIM_DEF,ID_NO,ACC_CLASS,ISIC_NEW_FINAL_BY_OIS,ISIC_GROUP_NEW_BY_OIS,COUNT_CIF_22,COUNT_CIF_ALL,COUNT_CIF_NPF_22,COUNT_CIF_NPF_ALL,ACC_OPEN_DATE,ACC_OPEN_DATE_FOR_OD,MAIN_ACC_LINK,TDR_CIF,TDR_TIMES,SIZE_FLAG,NPF_LEGACY,CIF_CREDIT_LIMIT,FIRST_NPF_START_DATE,NEW_FINANCIAL_CIF,BRANCH_PROVINCE,DIVISION,LATE_CHARGE_DUE,COLLATERAL_REQUIRED,MAIN_OCCUPATION,EDUCATION,AGE,INCOME_LEVEL,TERM_LOAN,BILL_OLDEST_DATE,NO_OF_DPD,COMMER_COMMIT_LINK1,CCL_OPENED_DATE,CCL_CREDIT_LIMIT,NPF_FLAG,YEAR_CONTRACT,SEX,SUB_TYPE,SUB_TYPE_DESC,MARKET_CODE,MARKET_DESC,DEALER_CODE,DEALER_DESC,ISIC_NEW_FINAL,ISIC_13GROUP,ISIC_5GROUP,NEW_SIZE_FLAG,PROF_DIV_RATE,RESTRUCTURE_FLAG,LOAN_PAYMENT_HOLIDAY
67577,2021-10-31,750928539.0,ยุภาวรรณ คำทา,19042299,1.0,สาขาคลองตัน,7200.0,สินเชื่อเพื่ออุปโภค,80000.0,80000.00,B1,CBS,พุทธ,2_อื่นๆ,NaN,1230300037861,1.0,241041,22.7,1.0,1.0,0.0,0.0,2021-10-19,19/10/2021,NaN,0.0,NaN,1Retail,NaN,80000.0,NaN,NEWCIF,กรุงเทพมหานคร,1 กรุงเทพมหานครและปริมณฑล,0.0,บัญชีสินเชื่อนี้ไม่มีหลักประกัน,002-พนักงานและลูกจ้างในหน่วยงานรัฐวิสาหกิจ,03-ปริญญาตรี,34.0,"3-15,000 - 29,999",36M,30/11/2021,0.0,19042299.0,19/10/2021,80000.0,1_PF,2021.0,F,20002.0,สินเชื่อวงเงินอเนกประสงค์,1023.0,สินเชื่อสวัสดิการ,335.0,สินเชื่อสวัสดิการ MOU หน่วยงานราชการและรัฐวิสา...,241026,12.4,6.0,1-Retail/Consumer,8.0,0.0,0.0
135204,2021-11-30,750928539.0,ยุภาวรรณ คำทา,19042299,1.0,สาขาคลองตัน,7200.0,สินเชื่อเพื่ออุปโภค,80000.0,78048.77,B1,CBS,พุทธ,2_อื่นๆ,NaN,1230300037861,1.0,241041,22.7,1.0,1.0,0.0,0.0,2021-10-19,19/10/2021,NaN,0.0,NaN,1Retail,NaN,80000.0,NaN,NEWCIF,กรุงเทพมหานคร,1 กรุงเทพมหานครและปริมณฑล,0.0,บัญชีสินเชื่อนี้ไม่มีหลักประกัน,002-พนักงานและลูกจ้างในหน่วยงานรัฐวิสาหกิจ,03-ปริญญาตรี,34.0,"3-15,000 - 29,999",36M,4/1/2022,0.0,19042299.0,19/10/2021,80000.0,1_PF,2021.0,F,20002.0,สินเชื่อวงเงินอเนกประสงค์,1023.0,สินเชื่อสวัสดิการ,335.0,สินเชื่อสวัสดิการ MOU หน่วยงานราชการและรัฐวิสา...,241026,12.4,6.0,1-Retail/Consumer,8.0,0.0,0.0
203089,2021-12-31,750928539.0,ยุภาวรรณ คำทา,19042299,1.0,สาขาคลองตัน,7200.0,สินเชื่อเพื่ออุปโภค,80000.0,75944.86,B1,CBS,พุทธ,2_อื่นๆ,NaN,1230300037861,1.0,241041,22.7,1.0,1.0,0.0,0.0,2021-10-19,19/10/2021,NaN,0.0,NaN,1Retail,NaN,80000.0,NaN,NEWCIF,กรุงเทพมหานคร,1 กรุงเทพมหานครและปริมณฑล,0.0,บัญชีสินเชื่อนี้ไม่มีหลักประกัน,002-พนักงานและลูกจ้างในหน่วยงานรัฐวิสาหกิจ,03-ปริญญาตรี,34.0,"3-15,000 - 29,999",36M,31/1/2022,0.0,19042299.0,19/10/2021,80000.0,1_PF,2021.0,F,20002.0,สินเชื่อวงเงินอเนกประสงค์,1023.0,สินเชื่อสวัสดิการ,335.0,สินเชื่อสวัสดิการ MOU หน่วยงานราชการและรัฐวิสา...,241026,12.4,6.0,1-Retail/Consumer,8.0,0.0,0.0
272637,2022-01-01,750928539.0,ยุภาวรรณ คำทา,19042299,1.0,สาขาคลองตัน,7200.0,สินเชื่อเพื่ออุปโภค,80000.0,73877.51,B1,CBS,พุทธ,2_อื่นๆ,NaN,1230300037861,1.0,241041,22.7,1.0,1.0,0.0,0.0,2021-10-19,19/10/2021,NaN,0.0,NaN,1Retail,NaN,80000.0,NaN,OLDCIF,กรุงเทพมหานคร,1 กรุงเทพมหานครและปริมณฑล,0.0,บัญชีสินเชื่อนี้ไม่มีหลักประกัน,002-พนักงานและลูกจ้างในหน่วยงานรัฐวิสาหกิจ,03-ปริญญาตรี,34.0,"3-15,000 - 29,999",36M,28/2/2022,0.0,19042299.0,19/10/2021,80000.0,1_PF,2021.0,F,20002.0,สินเชื่อวงเงินอเนกประสงค์,1023.0,สินเชื่อสวัสดิการ,335.0,สินเชื่อสวัสดิการ MOU หน่วยงานราชการและรัฐวิสา...,241026,12.4,6.0,1-Retail/Consumer,8.0,0.0,0.0
342343,2022-02-28,750928539.0,ยุภาวรรณ คำทา,19042299,1.0,สาขาคลองตัน,7200.0,สินเชื่อเพื่ออุปโภค,80000.0,73877.51,B1,CBS,พุทธ,2_อื่นๆ,NaN,1230300037861,1.0,241041,22.7,1.0,1.0,0.0,0.0,2021-10-19,19/10/2021,NaN,0.0,NaN,1Retail,NaN,80000.0,NaN,OLDCIF,กรุงเทพมหานคร,1 กรุงเทพมหานครและปริมณฑล,0.0,บัญชีสินเชื่อนี้ไม่มีหลักประกัน,002-พนักงานและลูกจ้างในหน่วยงานรัฐวิสาหกิจ,03-ปริญญาตรี,34.0,"3-15,000 - 29,999",36M,28/2/2022,0.0,19042299.0,19/10/2021,80000.0,1_PF,2021.0,F,20002.0,สินเชื่อวงเงินอเนกประสงค์,1023.0,สินเชื่อสวัสดิการ,335.0,สินเชื่อสวัสดิการ MOU หน่วยงานราชการและรัฐวิสา...,241026,12.4,6.0,1-Retail/Consumer,8.0,0.0,0.0


In [18]:
cbs_data_con[cbs_data_con['ACC_ID'] == 19042299].shape

(15, 63)

In [19]:
cbs_data_con['ID_NO'].duplicated().sum()

33553

In [20]:
cbs_data_con['ACC_ID'].duplicated().sum()

31449

In [21]:
# unique all row

cbs_data_con.duplicated().sum()

0

In [22]:
cbs_data_con['ACC_ID'].isna().sum()

0

In [23]:
print(cbs_data_con['ACC_OPEN_DATE'].dtype)
print(cbs_data_con['AS_OF_DATE'].dtype)

datetime64[ns]
datetime64[ns]


# Booked date, MOB, YOB

In [24]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

def booked_date(df):
    # add field book mm/yyyy & yyyy/qq
    df['BOOKED_(MM/YYYY)'] = df['ACC_OPEN_DATE'].astype(str).str[5:7]  + "/" +  df['ACC_OPEN_DATE'].astype(str).str[0:4]
    df['BOOKED_(YYYY-QQ)'] = pd.to_datetime(df['BOOKED_(MM/YYYY)']).dt.to_period("Q")
    return df

def mob_yob(df):
    df['MOB'] = ((df['AS_OF_DATE'] - df['ACC_OPEN_DATE']) /np.timedelta64(1, 'M')).astype(int)
    df['YOB'] = ((df['AS_OF_DATE'] - df['ACC_OPEN_DATE']) /np.timedelta64(1, 'Y')).astype(int) 
    return df 

In [25]:
df = booked_date(cbs_data_con)
df = mob_yob(cbs_data_con)

In [26]:
df.head()

,AS_OF_DATE,CUST_ID,CUST_NAME,ACC_ID,BRANCH_CODE,BRANCH_DESC,PROD_TYPE,PROD_TYPE_DESC,CREDIT_LIMIT,LEDGER_BALANCE,CUST_CLASS,DATA_SOURCE,RELIGION,CORE_RELIGION,MUSLIM_DEF,ID_NO,ACC_CLASS,ISIC_NEW_FINAL_BY_OIS,ISIC_GROUP_NEW_BY_OIS,COUNT_CIF_22,COUNT_CIF_ALL,COUNT_CIF_NPF_22,COUNT_CIF_NPF_ALL,ACC_OPEN_DATE,ACC_OPEN_DATE_FOR_OD,MAIN_ACC_LINK,TDR_CIF,TDR_TIMES,SIZE_FLAG,NPF_LEGACY,CIF_CREDIT_LIMIT,FIRST_NPF_START_DATE,NEW_FINANCIAL_CIF,BRANCH_PROVINCE,DIVISION,LATE_CHARGE_DUE,COLLATERAL_REQUIRED,MAIN_OCCUPATION,EDUCATION,AGE,INCOME_LEVEL,TERM_LOAN,BILL_OLDEST_DATE,NO_OF_DPD,COMMER_COMMIT_LINK1,CCL_OPENED_DATE,CCL_CREDIT_LIMIT,NPF_FLAG,YEAR_CONTRACT,SEX,SUB_TYPE,SUB_TYPE_DESC,MARKET_CODE,MARKET_DESC,DEALER_CODE,DEALER_DESC,ISIC_NEW_FINAL,ISIC_13GROUP,ISIC_5GROUP,NEW_SIZE_FLAG,PROF_DIV_RATE,RESTRUCTURE_FLAG,LOAN_PAYMENT_HOLIDAY,BOOKED_(MM/YYYY),BOOKED_(YYYY-QQ),MOB,YOB
67577,2021-10-31,750928539.0,ยุภาวรรณ คำทา,19042299,1.0,สาขาคลองตัน,7200.0,สินเชื่อเพื่ออุปโภค,80000.0,80000.00,B1,CBS,พุทธ,2_อื่นๆ,NaN,1230300037861,1.0,241041,22.7,1.0,1.0,0.0,0.0,2021-10-19,19/10/2021,NaN,0.0,NaN,1Retail,NaN,80000.0,NaN,NEWCIF,กรุงเทพมหานคร,1 กรุงเทพมหานครและปริมณฑล,0.0,บัญชีสินเชื่อนี้ไม่มีหลักประกัน,002-พนักงานและลูกจ้างในหน่วยงานรัฐวิสาหกิจ,03-ปริญญาตรี,34.0,"3-15,000 - 29,999",36M,30/11/2021,0.0,19042299.0,19/10/2021,80000.0,1_PF,2021.0,F,20002.0,สินเชื่อวงเงินอเนกประสงค์,1023.0,สินเชื่อสวัสดิการ,335.0,สินเชื่อสวัสดิการ MOU หน่วยงานราชการและรัฐวิสา...,241026,12.4,6.0,1-Retail/Consumer,8.0,0.0,0.0,10/2021,2021Q4,0,0
135204,2021-11-30,750928539.0,ยุภาวรรณ คำทา,19042299,1.0,สาขาคลองตัน,7200.0,สินเชื่อเพื่ออุปโภค,80000.0,78048.77,B1,CBS,พุทธ,2_อื่นๆ,NaN,1230300037861,1.0,241041,22.7,1.0,1.0,0.0,0.0,2021-10-19,19/10/2021,NaN,0.0,NaN,1Retail,NaN,80000.0,NaN,NEWCIF,กรุงเทพมหานคร,1 กรุงเทพมหานครและปริมณฑล,0.0,บัญชีสินเชื่อนี้ไม่มีหลักประกัน,002-พนักงานและลูกจ้างในหน่วยงานรัฐวิสาหกิจ,03-ปริญญาตรี,34.0,"3-15,000 - 29,999",36M,4/1/2022,0.0,19042299.0,19/10/2021,80000.0,1_PF,2021.0,F,20002.0,สินเชื่อวงเงินอเนกประสงค์,1023.0,สินเชื่อสวัสดิการ,335.0,สินเชื่อสวัสดิการ MOU หน่วยงานราชการและรัฐวิสา...,241026,12.4,6.0,1-Retail/Consumer,8.0,0.0,0.0,10/2021,2021Q4,1,0
203089,2021-12-31,750928539.0,ยุภาวรรณ คำทา,19042299,1.0,สาขาคลองตัน,7200.0,สินเชื่อเพื่ออุปโภค,80000.0,75944.86,B1,CBS,พุทธ,2_อื่นๆ,NaN,1230300037861,1.0,241041,22.7,1.0,1.0,0.0,0.0,2021-10-19,19/10/2021,NaN,0.0,NaN,1Retail,NaN,80000.0,NaN,NEWCIF,กรุงเทพมหานคร,1 กรุงเทพมหานครและปริมณฑล,0.0,บัญชีสินเชื่อนี้ไม่มีหลักประกัน,002-พนักงานและลูกจ้างในหน่วยงานรัฐวิสาหกิจ,03-ปริญญาตรี,34.0,"3-15,000 - 29,999",36M,31/1/2022,0.0,19042299.0,19/10/2021,80000.0,1_PF,2021.0,F,20002.0,สินเชื่อวงเงินอเนกประสงค์,1023.0,สินเชื่อสวัสดิการ,335.0,สินเชื่อสวัสดิการ MOU หน่วยงานราชการและรัฐวิสา...,241026,12.4,6.0,1-Retail/Consumer,8.0,0.0,0.0,10/2021,2021Q4,2,0
272637,2022-01-01,750928539.0,ยุภาวรรณ คำทา,19042299,1.0,สาขาคลองตัน,7200.0,สินเชื่อเพื่ออุปโภค,80000.0,73877.51,B1,CBS,พุทธ,2_อื่นๆ,NaN,1230300037861,1.0,241041,22.7,1.0,1.0,0.0,0.0,2021-10-19,19/10/2021,NaN,0.0,NaN,1Retail,NaN,80000.0,NaN,OLDCIF,กรุงเทพมหานคร,1 กรุงเทพมหานครและปริมณฑล,0.0,บัญชีสินเชื่อนี้ไม่มีหลักประกัน,002-พนักงานและลูกจ้างในหน่วยงานรัฐวิสาหกิจ,03-ปริญญาตรี,34.0,"3-15,000 - 29,999",36M,28/2/2022,0.0,19042299.0,19/10/2021,80000.0,1_PF,2021.0,F,20002.0,สินเชื่อวงเงินอเนกประสงค์,1023.0,สินเชื่อสวัสดิการ,335.0,สินเชื่อสวัสดิการ MOU หน่วยงานราชการและรัฐวิสา...,241026,12.4,6.0,1-Retail/Consumer,8.0,0.0,0.0,10/2021,2021Q4,2,0
342343,2022-02-28,750928539.0,ยุภาวรรณ คำทา,19042299,1.0,สาขาคลองตัน,7200.0,สินเชื่อเพื่ออุปโภค,80000.0,73877.51,B1,CBS,พุทธ,2_อื่นๆ,NaN,1230300037861,1.0,241041,22.7,1.0,1.0,0.0,0.0,2021-10-19,19/10/2021,NaN,0.0,NaN,1Retail,NaN,80000.0,NaN,OLDCIF,กรุงเทพมหานคร,1 กรุงเทพมหานครและปริมณฑล,0.0,บัญชีสินเชื่อนี้ไม่มีหลักประกัน,002-พนักงานและลูกจ้างในหน่วยงานรัฐวิสาหกิจ,03-ปริญญาตรี,34.0,"3-15,000 - 29,999",36M,28/2/2022,0.0,19042299.0,19/10/2021,80000.0,1_PF,2021.0,F,20002.0,สินเชื่อวงเงินอเนกประสงค์,1023.0,สินเชื่อสวัสดิการ,335.0,สินเ

In [27]:
df['MOB'].value_counts()

0     7148
1     5807
2     5174
3     4179
4     3533
5     2617
6     2194
7     1792
8     1543
9     1312
10     990
11     790
12     604
13     347
14     152
Name: MOB, dtype: int64

In [28]:
df['YOB'].value_counts()

0    37079
1     1103
Name: YOB, dtype: int64

In [29]:
# check result
df[df['ACC_ID'] == 19042299][['AS_OF_DATE', 'MOB', 'YOB']]

,AS_OF_DATE,MOB,YOB
67577,2021-10-31,0,0
135204,2021-11-30,1,0
203089,2021-12-31,2,0
272637,2022-01-01,2,0
342343,2022-02-28,4,0
409743,2022-03-31,5,0
477478,2022-04-30,6,0
545444,2022-05-31,7,0
613660,2022-06-30,8,0
682371,2022-07-30,9,0


# DQ

In [30]:
def convert_dq(x):
    if (x <= 7):
        return 'DQ_0'
    elif (x >= 8) & (x <= 30):
        return 'DQ_X'
    elif (x >= 31) & (x <= 60):
        return 'DQ_30'
    elif (x >= 61) & (x <= 90):
        return 'DQ_60'
    elif (x >= 91):
        return 'DQ_90'
        
def convert_code(x):
    if (x <= 7):
        return 0
    elif (x >= 8) & (x <= 30):
        return 1
    elif (x >= 31) & (x <= 60):
        return 2
    elif (x >= 61) & (x <= 90):
        return 3
    elif (x >= 91):
        return 4

In [31]:
df['DQ'] = df['NO_OF_DPD'].apply(convert_dq)
df['CODE'] = df['NO_OF_DPD'].apply(convert_code)

In [32]:
df['DQ'].value_counts()

DQ_0     38002
DQ_X        68
DQ_30       52
DQ_90       35
DQ_60       25
Name: DQ, dtype: int64

In [33]:
len(df['ACC_ID'].unique())

6733

In [34]:
df[df['ACC_ID'] == 89307844]['CODE']

22357     0
90088     0
158015    0
227789    0
297627    0
365649    0
433535    0
501620    1
570019    2
638805    3
708086    4
777727    4
847583    4
917761    4
988423    4
Name: CODE, dtype: int64

In [35]:
df[df['ACC_ID'] == 5889068407]['CODE']

203833     0
273392     0
343108     0
410508     0
478245     0
546214     0
614435     1
683147     2
752321     3
821583     3
891232     4
961334     4
1031867    4
Name: CODE, dtype: int64

In [36]:
df.head()

,AS_OF_DATE,CUST_ID,CUST_NAME,ACC_ID,BRANCH_CODE,BRANCH_DESC,PROD_TYPE,PROD_TYPE_DESC,CREDIT_LIMIT,LEDGER_BALANCE,CUST_CLASS,DATA_SOURCE,RELIGION,CORE_RELIGION,MUSLIM_DEF,ID_NO,ACC_CLASS,ISIC_NEW_FINAL_BY_OIS,ISIC_GROUP_NEW_BY_OIS,COUNT_CIF_22,COUNT_CIF_ALL,COUNT_CIF_NPF_22,COUNT_CIF_NPF_ALL,ACC_OPEN_DATE,ACC_OPEN_DATE_FOR_OD,MAIN_ACC_LINK,TDR_CIF,TDR_TIMES,SIZE_FLAG,NPF_LEGACY,CIF_CREDIT_LIMIT,FIRST_NPF_START_DATE,NEW_FINANCIAL_CIF,BRANCH_PROVINCE,DIVISION,LATE_CHARGE_DUE,COLLATERAL_REQUIRED,MAIN_OCCUPATION,EDUCATION,AGE,INCOME_LEVEL,TERM_LOAN,BILL_OLDEST_DATE,NO_OF_DPD,COMMER_COMMIT_LINK1,CCL_OPENED_DATE,CCL_CREDIT_LIMIT,NPF_FLAG,YEAR_CONTRACT,SEX,SUB_TYPE,SUB_TYPE_DESC,MARKET_CODE,MARKET_DESC,DEALER_CODE,DEALER_DESC,ISIC_NEW_FINAL,ISIC_13GROUP,ISIC_5GROUP,NEW_SIZE_FLAG,PROF_DIV_RATE,RESTRUCTURE_FLAG,LOAN_PAYMENT_HOLIDAY,BOOKED_(MM/YYYY),BOOKED_(YYYY-QQ),MOB,YOB,DQ,CODE
67577,2021-10-31,750928539.0,ยุภาวรรณ คำทา,19042299,1.0,สาขาคลองตัน,7200.0,สินเชื่อเพื่ออุปโภค,80000.0,80000.00,B1,CBS,พุทธ,2_อื่นๆ,NaN,1230300037861,1.0,241041,22.7,1.0,1.0,0.0,0.0,2021-10-19,19/10/2021,NaN,0.0,NaN,1Retail,NaN,80000.0,NaN,NEWCIF,กรุงเทพมหานคร,1 กรุงเทพมหานครและปริมณฑล,0.0,บัญชีสินเชื่อนี้ไม่มีหลักประกัน,002-พนักงานและลูกจ้างในหน่วยงานรัฐวิสาหกิจ,03-ปริญญาตรี,34.0,"3-15,000 - 29,999",36M,30/11/2021,0.0,19042299.0,19/10/2021,80000.0,1_PF,2021.0,F,20002.0,สินเชื่อวงเงินอเนกประสงค์,1023.0,สินเชื่อสวัสดิการ,335.0,สินเชื่อสวัสดิการ MOU หน่วยงานราชการและรัฐวิสา...,241026,12.4,6.0,1-Retail/Consumer,8.0,0.0,0.0,10/2021,2021Q4,0,0,DQ_0,0
135204,2021-11-30,750928539.0,ยุภาวรรณ คำทา,19042299,1.0,สาขาคลองตัน,7200.0,สินเชื่อเพื่ออุปโภค,80000.0,78048.77,B1,CBS,พุทธ,2_อื่นๆ,NaN,1230300037861,1.0,241041,22.7,1.0,1.0,0.0,0.0,2021-10-19,19/10/2021,NaN,0.0,NaN,1Retail,NaN,80000.0,NaN,NEWCIF,กรุงเทพมหานคร,1 กรุงเทพมหานครและปริมณฑล,0.0,บัญชีสินเชื่อนี้ไม่มีหลักประกัน,002-พนักงานและลูกจ้างในหน่วยงานรัฐวิสาหกิจ,03-ปริญญาตรี,34.0,"3-15,000 - 29,999",36M,4/1/2022,0.0,19042299.0,19/10/2021,80000.0,1_PF,2021.0,F,20002.0,สินเชื่อวงเงินอเนกประสงค์,1023.0,สินเชื่อสวัสดิการ,335.0,สินเชื่อสวัสดิการ MOU หน่วยงานราชการและรัฐวิสา...,241026,12.4,6.0,1-Retail/Consumer,8.0,0.0,0.0,10/2021,2021Q4,1,0,DQ_0,0
203089,2021-12-31,750928539.0,ยุภาวรรณ คำทา,19042299,1.0,สาขาคลองตัน,7200.0,สินเชื่อเพื่ออุปโภค,80000.0,75944.86,B1,CBS,พุทธ,2_อื่นๆ,NaN,1230300037861,1.0,241041,22.7,1.0,1.0,0.0,0.0,2021-10-19,19/10/2021,NaN,0.0,NaN,1Retail,NaN,80000.0,NaN,NEWCIF,กรุงเทพมหานคร,1 กรุงเทพมหานครและปริมณฑล,0.0,บัญชีสินเชื่อนี้ไม่มีหลักประกัน,002-พนักงานและลูกจ้างในหน่วยงานรัฐวิสาหกิจ,03-ปริญญาตรี,34.0,"3-15,000 - 29,999",36M,31/1/2022,0.0,19042299.0,19/10/2021,80000.0,1_PF,2021.0,F,20002.0,สินเชื่อวงเงินอเนกประสงค์,1023.0,สินเชื่อสวัสดิการ,335.0,สินเชื่อสวัสดิการ MOU หน่วยงานราชการและรัฐวิสา...,241026,12.4,6.0,1-Retail/Consumer,8.0,0.0,0.0,10/2021,2021Q4,2,0,DQ_0,0
272637,2022-01-01,750928539.0,ยุภาวรรณ คำทา,19042299,1.0,สาขาคลองตัน,7200.0,สินเชื่อเพื่ออุปโภค,80000.0,73877.51,B1,CBS,พุทธ,2_อื่นๆ,NaN,1230300037861,1.0,241041,22.7,1.0,1.0,0.0,0.0,2021-10-19,19/10/2021,NaN,0.0,NaN,1Retail,NaN,80000.0,NaN,OLDCIF,กรุงเทพมหานคร,1 กรุงเทพมหานครและปริมณฑล,0.0,บัญชีสินเชื่อนี้ไม่มีหลักประกัน,002-พนักงานและลูกจ้างในหน่วยงานรัฐวิสาหกิจ,03-ปริญญาตรี,34.0,"3-15,000 - 29,999",36M,28/2/2022,0.0,19042299.0,19/10/2021,80000.0,1_PF,2021.0,F,20002.0,สินเชื่อวงเงินอเนกประสงค์,1023.0,สินเชื่อสวัสดิการ,335.0,สินเชื่อสวัสดิการ MOU หน่วยงานราชการและรัฐวิสา...,241026,12.4,6.0,1-Retail/Consumer,8.0,0.0,0.0,10/2021,2021Q4,2,0,DQ_0,0
342343,2022-02-28,750928539.0,ยุภาวรรณ คำทา,19042299,1.0,สาขาคลองตัน,7200.0,สินเชื่อเพื่ออุปโภค,80000.0,73877.51,B1,CBS,พุทธ,2_อื่นๆ,NaN,1230300037861,1.0,241041,22.7,1.0,1.0,0.0,0.0,2021-10-19,19/10/2021,NaN,0.0,NaN,1Retail,NaN,80000.0,NaN,OLDCIF,กรุงเทพมหานคร,1 กรุงเทพมหานครและปริมณฑล,0.0,บัญชีสินเชื่อนี้ไม่มีหลักประกัน,002-พนักงานและลูกจ้างในหน่วยงานรัฐวิสาหกิจ,03-ปริญญาตรี,34.0,"3-15,000 - 29,999",36M,28/2/2022,0.0,19042299.0,19/10/2021,80000.0,1_PF,2021.0,F,20002.0,สินเชื่อวงเงินอเนกประสงค์

# EVER DQ

In [37]:
df2 = df.copy()

In [38]:
# test
aa = df2[(df2['ACC_ID'] == 89307844) | (df2['ACC_ID'] == 5889068407)]

In [39]:
print(aa.shape)
aa.head()

(28, 69)


,AS_OF_DATE,CUST_ID,CUST_NAME,ACC_ID,BRANCH_CODE,BRANCH_DESC,PROD_TYPE,PROD_TYPE_DESC,CREDIT_LIMIT,LEDGER_BALANCE,CUST_CLASS,DATA_SOURCE,RELIGION,CORE_RELIGION,MUSLIM_DEF,ID_NO,ACC_CLASS,ISIC_NEW_FINAL_BY_OIS,ISIC_GROUP_NEW_BY_OIS,COUNT_CIF_22,COUNT_CIF_ALL,COUNT_CIF_NPF_22,COUNT_CIF_NPF_ALL,ACC_OPEN_DATE,ACC_OPEN_DATE_FOR_OD,MAIN_ACC_LINK,TDR_CIF,TDR_TIMES,SIZE_FLAG,NPF_LEGACY,CIF_CREDIT_LIMIT,FIRST_NPF_START_DATE,NEW_FINANCIAL_CIF,BRANCH_PROVINCE,DIVISION,LATE_CHARGE_DUE,COLLATERAL_REQUIRED,MAIN_OCCUPATION,EDUCATION,AGE,INCOME_LEVEL,TERM_LOAN,BILL_OLDEST_DATE,NO_OF_DPD,COMMER_COMMIT_LINK1,CCL_OPENED_DATE,CCL_CREDIT_LIMIT,NPF_FLAG,YEAR_CONTRACT,SEX,SUB_TYPE,SUB_TYPE_DESC,MARKET_CODE,MARKET_DESC,DEALER_CODE,DEALER_DESC,ISIC_NEW_FINAL,ISIC_13GROUP,ISIC_5GROUP,NEW_SIZE_FLAG,PROF_DIV_RATE,RESTRUCTURE_FLAG,LOAN_PAYMENT_HOLIDAY,BOOKED_(MM/YYYY),BOOKED_(YYYY-QQ),MOB,YOB,DQ,CODE
22357,2021-10-31,750262093.0,บูกอรี มะเกะ,89307844,8.0,สาขาอโศก,7200.0,สินเชื่อเพื่ออุปโภค,90000.0,90000.00,B1,CBS,อิสลาม,1_มุสลิม,1,1960100139540,1.0,241041,22.7,1.0,1.0,0.0,0.0,2021-10-25,25/10/2021,NaN,0.0,NaN,1Retail,NaN,90000.0,NaN,NEWCIF,กรุงเทพมหานคร,1 กรุงเทพมหานครและปริมณฑล,0.0,บัญชีสินเชื่อนี้ไม่มีหลักประกัน,003-พนักงานและลูกจ้างในบริษัท,03-ปริญญาตรี,26.0,"1-<10,000",84M,30/11/2021,0.0,89307844.0,25/10/2021,90000.0,1_PF,2021.0,M,20002.0,สินเชื่อวงเงินอเนกประสงค์,1023.0,สินเชื่อสวัสดิการ,334.0,สินเชื่อสวัสดิการ MOU บริษัทจดทะเบียนในตลาดหลั...,241026,12.4,6.0,1-Retail/Consumer,8.75,0.0,0.0,10/2021,2021Q4,0,0,DQ_0,0
90088,2021-11-30,750262093.0,บูกอรี มะเกะ,89307844,8.0,สาขาอโศก,7200.0,สินเชื่อเพื่ออุปโภค,90000.0,90000.00,B1,CBS,อิสลาม,1_มุสลิม,1,1960100139540,1.0,241041,22.7,1.0,1.0,0.0,0.0,2021-10-25,25/10/2021,NaN,0.0,NaN,1Retail,NaN,90000.0,NaN,NEWCIF,กรุงเทพมหานคร,1 กรุงเทพมหานครและปริมณฑล,0.0,บัญชีสินเชื่อนี้ไม่มีหลักประกัน,003-พนักงานและลูกจ้างในบริษัท,03-ปริญญาตรี,26.0,"1-<10,000",84M,30/11/2021,0.0,89307844.0,25/10/2021,90000.0,1_PF,2021.0,M,20002.0,สินเชื่อวงเงินอเนกประสงค์,1023.0,สินเชื่อสวัสดิการ,334.0,สินเชื่อสวัสดิการ MOU บริษัทจดทะเบียนในตลาดหลั...,241026,12.4,6.0,1-Retail/Consumer,8.75,0.0,0.0,10/2021,2021Q4,1,0,DQ_0,0
158015,2021-12-31,750262093.0,บูกอรี มะเกะ,89307844,8.0,สาขาอโศก,7200.0,สินเชื่อเพื่ออุปโภค,90000.0,89276.71,B1,CBS,อิสลาม,1_มุสลิม,1,1960100139540,1.0,241041,22.7,1.0,1.0,0.0,0.0,2021-10-25,25/10/2021,NaN,0.0,NaN,1Retail,NaN,90000.0,NaN,NEWCIF,กรุงเทพมหานคร,1 กรุงเทพมหานครและปริมณฑล,0.0,บัญชีสินเชื่อนี้ไม่มีหลักประกัน,003-พนักงานและลูกจ้างในบริษัท,03-ปริญญาตรี,26.0,"1-<10,000",84M,4/1/2022,0.0,89307844.0,25/10/2021,90000.0,1_PF,2021.0,M,20002.0,สินเชื่อวงเงินอเนกประสงค์,1023.0,สินเชื่อสวัสดิการ,334.0,สินเชื่อสวัสดิการ MOU บริษัทจดทะเบียนในตลาดหลั...,241026,12.4,6.0,1-Retail/Consumer,8.75,0.0,0.0,10/2021,2021Q4,2,0,DQ_0,0
227789,2022-01-01,750262093.0,บูกอรี มะเกะ,89307844,8.0,สาขาอโศก,7200.0,สินเชื่อเพื่ออุปโภค,90000.0,87425.37,B1,CBS,อิสลาม,1_มุสลิม,1,1960100139540,1.0,241041,22.7,1.0,1.0,0.0,0.0,2021-10-25,25/10/2021,NaN,0.0,NaN,1Retail,NaN,90000.0,NaN,OLDCIF,กรุงเทพมหานคร,1 กรุงเทพมหานครและปริมณฑล,0.0,บัญชีสินเชื่อนี้ไม่มีหลักประกัน,003-พนักงานและลูกจ้างในบริษัท,03-ปริญญาตรี,26.0,"1-<10,000",84M,28/2/2022,0.0,89307844.0,25/10/2021,90000.0,1_PF,2021.0,M,20002.0,สินเชื่อวงเงินอเนกประสงค์,1023.0,สินเชื่อสวัสดิการ,85.0,ลูกค้าสินเชื่อสวัสดิการพนักงานพ้นสภาพ,241026,12.4,6.0,1-Retail/Consumer,16.00,0.0,0.0,10/2021,2021Q4,2,0,DQ_0,0
297627,2022-02-28,750262093.0,บูกอรี มะเกะ,89307844,8.0,สาขาอโศก,7200.0,สินเชื่อเพื่ออุปโภค,90000.0,87425.37,B1,CBS,อิสลาม,1_มุสลิม,1,1960100139540,1.0,241041,22.7,1.0,1.0,0.0,0.0,2021-10-25,25/10/2021,NaN,0.0,NaN,1Retail,NaN,90000.0,NaN,OLDCIF,กรุงเทพมหานคร,1 กรุงเทพมหานครและปริมณฑล,0.0,บัญชีสินเชื่อนี้ไม่มีหลักประกัน,003-พนักงานและลูกจ้างในบริษัท,03-ปริญญาตรี,26.0,"1-<10,000",84M,28/2/2022,0.0,89307844.0,25/10/2021,90000.0,1_PF,2021.0,M,20002.0,สินเชื่อวงเงินอเนกประสงค์,1023.0,สินเชื่อสวัสดิการ,85.0,ลูกค้าสินเชื่อสวัสดิการพนักงานพ้นสภาพ,241026,12.4,6.0,1-Retail/Consumer,16.00,0.0,0.0,10/2021,202

In [40]:
def ever_dq(df):
    # find account number 
    acc = df['ACC_ID'].drop_duplicates()

    # create field EVER_DQ_X, 30, 60, 90
    df['EVER_DQ_X'] = np.nan
    df['EVER_DQ_30'] = np.nan
    df['EVER_DQ_60'] = np.nan
    df['EVER_DQ_90'] = np.nan

    newdf_list = []
    for a in acc:
        new_df = df[df['ACC_ID'].isin([a])]
        new_df.reset_index(drop=True, inplace=True)

        """ procedure for flag EVER_DQ"""

        """ DQ_X"""
        try:
            # find row index 
            rowindex_code1 = new_df[new_df['CODE'] == 1].index
            # find column index
            col_everdqx = [col for col in new_df.columns.to_list() if col[0:9] == 'EVER_DQ_X']
            ind_col_everdqx = [new_df.columns.get_loc(mon) for mon in col_everdqx]

            #flag ever dq_x
            new_df.iloc[rowindex_code1[0]:, ind_col_everdqx] = 1
        except:IndexError

        try:
            """ DQ_30"""
            # find row index 
            rowindex_code1 = new_df[new_df['CODE'] == 2].index
            # find column index
            col_everdqx = [col for col in new_df.columns.to_list() if col[0:10] == 'EVER_DQ_30']
            ind_col_everdqx = [new_df.columns.get_loc(mon) for mon in col_everdqx]

            #flag ever dq_x
            new_df.iloc[rowindex_code1[0]:, ind_col_everdqx] = 1
        except:IndexError
        
        try:
            """ DQ_60"""
            # find row index 
            rowindex_code1 = new_df[new_df['CODE'] == 3].index
            # find column index
            col_everdqx = [col for col in new_df.columns.to_list() if col[0:10] == 'EVER_DQ_60']
            ind_col_everdqx = [new_df.columns.get_loc(mon) for mon in col_everdqx]

            #flag ever dq_x
            new_df.iloc[rowindex_code1[0]:, ind_col_everdqx] = 1
        except:IndexError

        try:
            """ DQ_90"""
            # find row index 
            rowindex_code1 = new_df[new_df['CODE'] == 4].index
            # find column index
            col_everdqx = [col for col in new_df.columns.to_list() if col[0:10] == 'EVER_DQ_90']
            ind_col_everdqx = [new_df.columns.get_loc(mon) for mon in col_everdqx]

            #flag ever dq_x
            new_df.iloc[rowindex_code1[0]:, ind_col_everdqx] = 1
        except:IndexError

        # append to list
        newdf_list.append(new_df)
    
    return newdf_list

In [41]:
"""test few sample"""
# list_df = ever_dq(aa)
# pd.concat(list_df)

C:\Users\650010\AppData\Local\Temp\ipykernel_15936\4045629706.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['EVER_DQ_X'] = np.nan
C:\Users\650010\AppData\Local\Temp\ipykernel_15936\4045629706.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['EVER_DQ_30'] = np.nan
C:\Users\650010\AppData\Local\Temp\ipykernel_15936\4045629706.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

,AS_OF_DATE,CUST_ID,CUST_NAME,ACC_ID,BRANCH_CODE,BRANCH_DESC,PROD_TYPE,PROD_TYPE_DESC,CREDIT_LIMIT,LEDGER_BALANCE,CUST_CLASS,DATA_SOURCE,RELIGION,CORE_RELIGION,MUSLIM_DEF,ID_NO,ACC_CLASS,ISIC_NEW_FINAL_BY_OIS,ISIC_GROUP_NEW_BY_OIS,COUNT_CIF_22,COUNT_CIF_ALL,COUNT_CIF_NPF_22,COUNT_CIF_NPF_ALL,ACC_OPEN_DATE,ACC_OPEN_DATE_FOR_OD,MAIN_ACC_LINK,TDR_CIF,TDR_TIMES,SIZE_FLAG,NPF_LEGACY,CIF_CREDIT_LIMIT,FIRST_NPF_START_DATE,NEW_FINANCIAL_CIF,BRANCH_PROVINCE,DIVISION,LATE_CHARGE_DUE,COLLATERAL_REQUIRED,MAIN_OCCUPATION,EDUCATION,AGE,INCOME_LEVEL,TERM_LOAN,BILL_OLDEST_DATE,NO_OF_DPD,COMMER_COMMIT_LINK1,CCL_OPENED_DATE,CCL_CREDIT_LIMIT,NPF_FLAG,YEAR_CONTRACT,SEX,SUB_TYPE,SUB_TYPE_DESC,MARKET_CODE,MARKET_DESC,DEALER_CODE,DEALER_DESC,ISIC_NEW_FINAL,ISIC_13GROUP,ISIC_5GROUP,NEW_SIZE_FLAG,PROF_DIV_RATE,RESTRUCTURE_FLAG,LOAN_PAYMENT_HOLIDAY,BOOKED_(MM/YYYY),BOOKED_(YYYY-QQ),MOB,YOB,DQ,CODE,EVER_DQ_X,EVER_DQ_30,EVER_DQ_60,EVER_DQ_90
0,2021-10-31,750262093.0,บูกอรี มะเกะ,89307844,8.0,สาขาอโศก,7200.0,สินเชื่อเพื่ออุปโภค,90000.0,90000.00,B1,CBS,อิสลาม,1_มุสลิม,1,1960100139540,1.0,241041,22.7,1.0,1.0,0.0,0.0,2021-10-25,25/10/2021,NaN,0.0,NaN,1Retail,NaN,90000.0,NaN,NEWCIF,กรุงเทพมหานคร,1 กรุงเทพมหานครและปริมณฑล,0.00,บัญชีสินเชื่อนี้ไม่มีหลักประกัน,003-พนักงานและลูกจ้างในบริษัท,03-ปริญญาตรี,26.0,"1-<10,000",84M,30/11/2021,0.0,8.930784e+07,25/10/2021,90000.0,1_PF,2021.0,M,20002.0,สินเชื่อวงเงินอเนกประสงค์,1023.0,สินเชื่อสวัสดิการ,334.0,สินเชื่อสวัสดิการ MOU บริษัทจดทะเบียนในตลาดหลั...,241026,12.4,6.0,1-Retail/Consumer,8.75,0.0,0.0,10/2021,2021Q4,0,0,DQ_0,0,NaN,NaN,NaN,NaN
1,2021-11-30,750262093.0,บูกอรี มะเกะ,89307844,8.0,สาขาอโศก,7200.0,สินเชื่อเพื่ออุปโภค,90000.0,90000.00,B1,CBS,อิสลาม,1_มุสลิม,1,1960100139540,1.0,241041,22.7,1.0,1.0,0.0,0.0,2021-10-25,25/10/2021,NaN,0.0,NaN,1Retail,NaN,90000.0,NaN,NEWCIF,กรุงเทพมหานคร,1 กรุงเทพมหานครและปริมณฑล,0.00,บัญชีสินเชื่อนี้ไม่มีหลักประกัน,003-พนักงานและลูกจ้างในบริษัท,03-ปริญญาตรี,26.0,"1-<10,000",84M,30/11/2021,0.0,8.930784e+07,25/10/2021,90000.0,1_PF,2021.0,M,20002.0,สินเชื่อวงเงินอเนกประสงค์,1023.0,สินเชื่อสวัสดิการ,334.0,สินเชื่อสวัสดิการ MOU บริษัทจดทะเบียนในตลาดหลั...,241026,12.4,6.0,1-Retail/Consumer,8.75,0.0,0.0,10/2021,2021Q4,1,0,DQ_0,0,NaN,NaN,NaN,NaN
2,2021-12-31,750262093.0,บูกอรี มะเกะ,89307844,8.0,สาขาอโศก,7200.0,สินเชื่อเพื่ออุปโภค,90000.0,89276.71,B1,CBS,อิสลาม,1_มุสลิม,1,1960100139540,1.0,241041,22.7,1.0,1.0,0.0,0.0,2021-10-25,25/10/2021,NaN,0.0,NaN,1Retail,NaN,90000.0,NaN,NEWCIF,กรุงเทพมหานคร,1 กรุงเทพมหานครและปริมณฑล,0.00,บัญชีสินเชื่อนี้ไม่มีหลักประกัน,003-พนักงานและลูกจ้างในบริษัท,03-ปริญญาตรี,26.0,"1-<10,000",84M,4/1/2022,0.0,8.930784e+07,25/10/2021,90000.0,1_PF,2021.0,M,20002.0,สินเชื่อวงเงินอเนกประสงค์,1023.0,สินเชื่อสวัสดิการ,334.0,สินเชื่อสวัสดิการ MOU บริษัทจดทะเบียนในตลาดหลั...,241026,12.4,6.0,1-Retail/Consumer,8.75,0.0,0.0,10/2021,2021Q4,2,0,DQ_0,0,NaN,NaN,NaN,NaN
3,2022-01-01,750262093.0,บูกอรี มะเกะ,89307844,8.0,สาขาอโศก,7200.0,สินเชื่อเพื่ออุปโภค,90000.0,87425.37,B1,CBS,อิสลาม,1_มุสลิม,1,1960100139540,1.0,241041,22.7,1.0,1.0,0.0,0.0,2021-10-25,25/10/2021,NaN,0.0,NaN,1Retail,NaN,90000.0,NaN,OLDCIF,กรุงเทพมหานคร,1 กรุงเทพมหานครและปริมณฑล,0.00,บัญชีสินเชื่อนี้ไม่มีหลักประกัน,003-พนักงานและลูกจ้างในบริษัท,03-ปริญญาตรี,26.0,"1-<10,000",84M,28/2/2022,0.0,8.930784e+07,25/10/2021,90000.0,1_PF,2021.0,M,20002.0,สินเชื่อวงเงินอเนกประสงค์,1023.0,สินเชื่อสวัสดิการ,85.0,ลูกค้าสินเชื่อสวัสดิการพนักงานพ้นสภาพ,241026,12.4,6.0,1-Retail/Consumer,16.00,0.0,0.0,10/2021,2021Q4,2,0,DQ_0,0,NaN,NaN,NaN,NaN
4,2022-02-28,750262093.0,บูกอรี มะเกะ,89307844,8.0,สาขาอโศก,7200.0,สินเชื่อเพื่ออุปโภค,90000.0,87425.37,B1,CBS,อิสลาม,1_มุสลิม,1,1960100139540,1.0,241041,22.7,1.0,1.0,0.0,0.0,2021-10-25,25/10/2021,NaN,0.0,NaN,1Retail,NaN,90000.0,NaN,OLDCIF,กรุงเทพมหานคร,1 กรุงเทพมหานครและปริมณฑล,0.00,บัญชีสินเชื่อนี้ไม่มีหลักประกัน,003-พนักงานและลูกจ้างในบริษัท,03-ปริญญาตรี,26.0,"1-<10,000",84M,28/2/2022,0.0,8.930784e+07,25/10/2021,90000.0,1_PF,2021.0,M,20002.0,สินเชื่อวงเงินอเนกประสงค์,1023.0,สินเชื่อสวัสดิการ,85.

In [42]:
"""all data"""
list_df3 = ever_dq(df2)

""" concat data from list"""
df3 = pd.concat(list_df3)

In [56]:
# visual data field: ever_dq
df3[df3['ACC_ID'] == 89307844][['AS_OF_DATE', 'ACC_ID', 'DQ','EVER_DQ_X', 'EVER_DQ_30', 'EVER_DQ_60', 'EVER_DQ_90']]

,AS_OF_DATE,ACC_ID,DQ,EVER_DQ_X,EVER_DQ_30,EVER_DQ_60,EVER_DQ_90
0,31/10/2021,89307844,DQ_0,NaN,NaN,NaN,NaN
1,30/11/2021,89307844,DQ_0,NaN,NaN,NaN,NaN
2,31/12/2021,89307844,DQ_0,NaN,NaN,NaN,NaN
3,01/01/2022,89307844,DQ_0,NaN,NaN,NaN,NaN
4,28/02/2022,89307844,DQ_0,NaN,NaN,NaN,NaN
5,31/03/2022,89307844,DQ_0,NaN,NaN,NaN,NaN
6,30/04/2022,89307844,DQ_0,NaN,NaN,NaN,NaN
7,31/05/2022,89307844,DQ_X,1.0,NaN,NaN,NaN
8,30/06/2022,89307844,DQ_30,1.0,1.0,NaN,NaN
9,30/07/2022,89307844,DQ_60,1.0,1.0,1.0,NaN


# Change format for export 

In [44]:
df3['ACC_OPEN_DATE'] = df3['ACC_OPEN_DATE'].dt.strftime('%d/%m/%Y')
df3['AS_OF_DATE'] = df3['AS_OF_DATE'].dt.strftime('%d/%m/%Y')

# Export data

In [45]:
# df3.to_excel(output+"CAP_CBS_Performace_2.1.xlsx", index=False, encoding="TIS-620")
# df3.to_csv(output+"CAP_CBS_Performace_2.txt", index=False, encoding="TIS-620", sep="|")